# AV Calculator Python

Things I don't understand:
- why is tier here separated into 1 or 2? This is different than the metal plans. So then why is the 1 or 2 used to calculate tier specific inputs?
- csr levels (cost sharing reduction)
- in avc_1_main, why is there a for loop causing it to run twice?

Methods:
- avc_1_main
- avc_2-1_combined
- avc_2-2_sep_deduct_comb_moop
- avc_2-3_sep_deduct_sep_moop
- avc_2_globals
- avc_3_on_click
- avc_4_validate
- avc_5_formatting
- avc_6_create_image
- avc_7_initialize
- avc_8_functions
- avc_9_plan_data
- avc_10_process_plan_data
- avc_11_set_cont_tables
- avc_12_hsa
- avc_13_reset_vars
- avc_14_calc_av
- avc_15_process_av

In [3]:
class AV_Calculator:
    def __init__(self): 
        self.avc_2_globals()
    
    def avc_1_main(self):
        av_1 = 0
        av_2 = 0
        self.avc_9_plan_data()
        self.avc_4_validate()    
        # I am not sure why this runs twice, but it is this way in the Excel macros.
        for x in range(1,2):
            if self.gCB_MULTITIER_PLAN == 1: # if it's a multitier plan, calculate the weighted average of the AV's
                self.get_cost_sharing_parameters(1)
                self.avc_4_validate(1)
                av_1 = self.avc_14_calc_av(1)
                self.get_cost_sharing_parameters(2)
                self.avc_4_validate(2)
                av_2 = self.avc_14_calc_av(2)
                av = tier_1_util * av_1 + tier_2_util * av_2   
            else: # otherwise, just calculate the first AV
                self.get_cost_sharing_parameters(1)
                self.validate_tier_specific_inputs(1)
                av = self.avc_14_calc_av(tier_1)          
            if hsa_index == 1: # adjust for the hsa index (?)
                av = self.avc_12_hsa()
            av = min(1, av)  # cap AV at 1, make sure that AV is valid
            metal = self.avc_15_process_av(av)
            if metal == desired_tier:
                break
            else:
                print("desired tier does not match with calculated metal tier")
                
    def avc_21_combined(self):
        pass
    
    def avc_22_sep_deduct_comb_moop:
        pass
    
    def avc_23_sep_deduct_sep_moop:
        pass
    
    # define all of the global variables
    def avc_2_globals:
        self.avc_13_reset_vars()
    
    def avc_3_on_click:
        pass
    
    def avc_4_validate:
        pass
    
    def avc_5_formatting:
        pass
    
    def avc_6_create_image:
        pass
    
    def avc_7_initialize:
        pass
    
    def avc_8_functions:
        pass
    
    def avc_9_plan_data:
        pass
    
    def avc_10_process_plan_data:
        pass
    
    def avc_11_set_cont_tables:
        pass
    
    #hsa stands for health savings account i think
    def avc_12_hsa(self):
        hsa_row = 0
        if benecoins_deduct != 0:
            row = hsa_contrbtn / benecoins_deduct
        
        # get correct hsa row from table 
        #HSA_VALUE = computeRowVal(gMaxdCol.Rng(1), HSA_ROW) * gBENECOINS_DEDUCT
        #AV_DENOM = gMaxdCol.Rng(1).Cells(gBottomRow, 1)
        
        # then do math
        #BENEPAY = (1 - vAV) * AV_DENOM
        #HSA_ADJ = Application.Min(BENEPAY, HSA_VALUE)   
        #HSA_MOD_AV = (vAV * AV_DENOM + HSA_ADJ) / AV_DENOM
        #HSA_ADJUSTMENT = HSA_MOD_AV
        
    
        
    def avc_14_calc_av(self, tier):
        temp = 0
        get_cost_sharing_parameters() # redundant call, appears in excel macros
        # depending on deductibles and max out of pocket payment, calls different methods to use the 
        # cost sharing parameters and tables given in the excel spreadsheet to calculate AV
        SetContTable_Combined
    SetContTable_Med_and_Rx
    
    If gCB_INT_DEDUCT_IND = 1 Then
    
        CombinedMacro_v2
        CombinedMacro2_v2
        
    ElseIf gCB_INT_DEDUCT_IND = 0 And gCB_SEP_MOOP_IND = 0 Then
    
        SepDeductCombinedMOOP_v2
        SepDeductCombinedMOOP2_v2
        
    ElseIf gCB_INT_DEDUCT_IND = 0 And gCB_SEP_MOOP_IND = 1 Then
        
        SepMacro_v2
        SepMacro2_v2

    End If
       
    CalcAV = gAV
        return temp       
    
    
    
    def avc_15_process_av(self, csr3, csr2, csr1):    
        if self.gCB_CSR_IND = 0:
            if 0.88 <= self.gAV < 0.92:
                self.gACTUAL_MTIER_NUM = 1
                self.gMTIER_NAME = "Platinum"
                return "platinum"
            elif 0.78 <= self.gAV < 0.82:
                self.gACTUAL_MTIER_NUM = 2
                self.gMTIER_NAME = "Gold"
                return "gold"
            elif 0.68 <= self.gAV < 0.72:
                self.gACTUAL_MTIER_NUM = 3
                self.gMTIER_NAME = "Silver"
                return "silver"
            elif 0.58 <= self.gAV < 0.62:
                self.gACTUAL_MTIER_NUM = 4
                self.gMTIER_NAME = "Bronze"
                return "bronze"
            else:
                self.gACTUAL_MTIER_NUM = -1
                self.gMTIER_NAME = ""
                print("Result is outside of +/- 2 percent de minimis variation")
                if 0.93 <= self.gAV < 0.95 and (csr3 or desired_tier == "platinum"):
                    self.gACTUAL_MTIER_NUM = 1
                    self.gMTIER_NAME = "Platinum"
                    metal = "platinum"
                    print("CSR level 94% (100-150% FPL)")
                elif 0.86 <= self.gAV < 0.88 and (csr2 or desired_tier == "gold"):
                    self.gACTUAL_MTIER_NUM = 2
                    self.gMTIER_NAME = "Gold"
                    metal = "gold"
                    print("CSR level 87% (150-200% FPL)")
                elif 0.72 <= self.gAV < 0.74 and (csr1 or desired_tier == "silver"):
                    self.gACTUAL_MTIER_NUM = 3
                    self.gMTIER_NAME = "Silver"
                    metal = "silver"
                    print("CSR Level of 73% (200-250% FPL)")
                elif 0.62 <= self.gAV < 0.65 and (expanded_bronze or desired_tier == "bronze"):
                    self.gACTUAL_MTIER_NUM = 4
                    self.gMTIER_NAME = "Bronze"
                    metal = "bronze"
                    print("Expanded Bronze Standard, 58% to 65%")
                else:
                    raise Exception("Plan does not match up to variation")
                return metal
            return "no_valid_av"
    
    # sets all variables to 0 or null (0 for simplification here)
    def avc_13_reset_vars(self):
        self.gContTableCombined = 0
        self.gContTableMed = 0
        self.gContTableRx = 0
        self.gDEDROW = 0
        self.gDEDROW_MED = 0
        self.gDEDROW_RX = 0
        self.gNoteMsg = 0
        self.gUpToCol = 0
        self.gMaxdCol = 0
        self.gER = 0
        self.gIP = 0
        self.gPC = 0
        self.gPCN = 0
        self.gSP = 0
        self.gPSY = 0
        self.gIMG = 0
        self.gST = 0
        self.gOT = 0
        self.gPREV = 0
        self.gLAB = 0
        self.gSNF = 0
        self.gUNCL = 0
        self.gXRAY = 0
        self.gXRAY_PC = 0
        self.gXRAY_SP = 0
        self.gXRAY_UNC = 0
        self.gOPFAC_PSY = 0
        self.gOPFAC_IMG = 0
        self.gOPFAC_ST = 0
        self.gOPFAC_OT = 0
        self.gOPFAC_LAB = 0
        self.gOPFAC_UNC = 0
        self.gOPPROF_PSY = 0
        self.gOPPROF_IMG = 0
        self.gOPPROF_ST = 0
        self.gOPPROF_OT = 0
        self.gOPPROF_LAB = 0
        self.gOPPROF_SRGRY = 0
        self.gRXGEN = 0
        self.gRXFORM = 0
        self.gRXNONFORM = 0
        self.gRXSPCLTY = 0
        self.gUpToCol_MEDTBL = 0
        self.gMaxdCol_MEDTBL = 0
        self.gER_MEDTBL = 0
        self.gIP_MEDTBL = 0
        self.gPC_MEDTBL = 0
        self.gPCN_MEDTBL = 0
        self.gSP_MEDTBL = 0
        self.gPSY_MEDTBL = 0
        self.gIMG_MEDTBL = 0
        self.gST_MEDTBL = 0
        self.gOT_MEDTBL = 0
        self.gPREV_MEDTBL = 0
        self.gLAB_MEDTBL = 0
        self.gSNF_MEDTBL = 0
        self.gUNCL_MEDTBL = 0
        self.gXRAY_MEDTBL = 0
        self.gXRAY_PC_MEDTBL = 0
        self.gXRAY_SP_MEDTBL = 0
        self.gXRAY_UNC_MEDTBL = 0
        self.gOPFAC_PSY_MEDTBL = 0
        self.gOPFAC_IMG_MEDTBL = 0
        self.gOPFAC_ST_MEDTBL = 0
        self.gOPFAC_OT_MEDTBL = 0
        self.gOPFAC_LAB_MEDTBL = 0
        self.gOPFAC_UNC_MEDTBL = 0
        self.gOPPROF_PSY_MEDTBL = 0
        self.gOPPROF_IMG_MEDTBL = 0
        self.gOPPROF_ST_MEDTBL = 0
        self.gOPPROF_OT_MEDTBL = 0
        self.gOPPROF_LAB_MEDTBL = 0
        self.gOPPROF_SRGRY_MEDTBL = 0
        self.gUpToCol_RXTBL = 0
        self.gMaxdCol_RXTBL = 0
        self.gRXGEN_RXTBL = 0
        self.gRXFORM_RXTBL = 0
        self.gRXNONFORM_RXTBL = 0
        self.gRXSPCLTY_RXTBL = 0
        self.gCombinedLabels_Arr = 0 
        self.gMedicalLabels_Arr = 0
        self.gRxLabels_Arr = 0
        self.gMedical_Arr = 0
        self.gRx_Arr = 0
        self.gMedSvcCopayNR_Arr = 0
        self.gMedSvcCoinsNR_Arr = 0
        self.gRxSvcCopayNR_Arr = 0
        self.gRxSvcCoinsNR_Arr = 0
        self.gSTDED_ARR = 0
        self.gSTCOINS_ARR = 0
        self.gOMIT_ARR = 0
        self.gSTDEDRX_ARR = 0
        self.gSTCOINSRX_ARR = 0
        self.gOMITRX_ARR = 0
        self.gSTATUS_MSG = 0
        self.gERROR_FLAG = 0
        self.gOPFAC_PSY_OVRD = 0
        self.gOPFAC_IMG_OVRD = 0
        self.gOPFAC_ST_OVRD = 0
        self.gOPFAC_OT_OVRD = 0
        self.gOPFAC_LAB_OVRD = 0
        self.gOPPROF_PSY_OVRD = 0
        self.gOPPROF_IMG_OVRD = 0
        self.gOPPROF_ST_OVRD = 0
        self.gOPPROF_OT_OVRD = 0
        self.gOPPROF_LAB_OVRD = 0
        self.gNOTE_XRAY_OVRD = 0
        self.gNOTE_OP_OVRD = 0
        self.gAV = 0
        self.gBENEPAY_COINSRNG = 0
        self.gNESTED_DEDUCT_IND = 0
        self.gPLAN_PAY = 0
        self.gPLAN_PAY_COINSRNG = 0
        self.gTOT_PAY = 0
        self.gTOT_PAY_MED = 0
        self.gTOT_PAY_RX = 0
        self.gBENEPAY_DEDUCT_MED = 0
        self.gBENEPAY_DEDUCT_RX = 0
        self.gBENECOINS_DEDUCT = 0
        self.gDEDUCT_ADJMT = 0
        self.gMOOP_ADJMT = 0
        self.gMOOP_ADJMT_MED = 0
        self.gMOOP_ADJMT_RX = 0
        self.gADJ_DEDUCT = 0
        self.gADJ_DEDUCT_MED = 0
        self.gADJ_DEDUCT_RX = 0
        self.gADJ_MOOP = 0
        self.gADJ_MOOP_MED = 0
        self.gADJ_MOOP_RX = 0
        self.gTROOP = 0
        self.gTROOP_MED = 0
        self.gTROOP_RX = 0
        self.gEFF_COINS = 0
        self.gEFF_COINS_MED = 0
        self.gEFF_COINS_RX = 0
        self.gEFF_COINS_NUM = 0
        self.gEFF_COINS_NUM_MED = 0
        self.gEFF_COINS_NUM_RX = 0
        self.gEFF_COINS_DENOM_MED = 0
        self.gEFF_COINS_DENOM_RX = 0
        self.gAV_DENOM = 0